<a href="https://colab.research.google.com/github/kknd4eva/azure_openai_function_calling/blob/main/azure_openai_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# grab the latest openai libraries
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.1 MB/s eta 0:00:00


In [19]:
import openai

## Set up our Azure OpenAI service
deployment_id = "shiny-new-model"
openai.api_type = "azure"
openai.api_key = ""
openai.api_base = ""
openai.api_version = "2023-07-01-preview"

In [31]:
# Lets create our function to use with OpenAI. In our use case, we want the model to be able to accurately calculate
# how many open ai bucks a customer is entitled to based on their loyalty tier.

def calculate_openai_bucks(tier, points) -> str:
    # defining the conversion rates for each tier
    tiers = {
        "bronze": 1,
        "silver": 2,
        "gold": 5
    }
    if tier.lower() not in tiers:
        return "Invalid tier. Please provide a valid tier: bronze, silver, or gold."
    if int(points) < 0:
        return "Invalid points. Please provide a positive number of points."

    openai_bucks = tiers[tier.lower()] * int(points)
    return "${:.2f}".format(openai_bucks)

# Let's also create a second function just to show that the model will choose the right function from those available
# based on the context provided in the conversation

def reverse_string(s):
    return s[::-1]



In [22]:
# lets test our functions first
display(calculate_openai_bucks("silver",100))
display(reverse_string("Show me the moeny!"))

'$200.00'

'!yneom eht em wohS'

In [33]:
# We need to create a description of our functions, a bit like a blueprint so that the model understands how to use them
functions=[
  {
    "name": "calculate_openai_bucks",
    "description": "Calculate the number of openai bucks a customer is entitled to based on their loyalty tier and the number of points they have",
    "parameters": {
      "type": "object",
      "properties": {
        "tier": {
          "type": "string",
          "enum": [
            "bronze",
            "silver",
            "gold"
          ],
          "description": "The loyalty tier of the customer"
        },
        "points": {
          "type": "string",
          "description": "The number of points the customer has"
        }
      },
      "required": [
        "tier",
        "points"
      ]
    }
  },
  {
    "name": "reverse_string",
    "description": "Takes a string and reverses all the characters, then returns the new reversed string",
    "parameters": {
      "type": "object",
      "properties": {
        "s": {
          "type": "string",
          "description": "The string to be reversed"
        }
      },
      "required": [
        "s"
      ]
    }
  }
]

available_functions = {
    "calculate_openai_bucks": calculate_openai_bucks,
    "reverse_string": reverse_string
}


In [24]:
# Sourced from https://github.com/Azure-Samples/openai/blob/main/Basic_Samples/Functions/working_with_functions.ipynb
# a helper method just to make sure that the functions are being used correctly
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [39]:
from openai.api_resources import model
import json # needed for this function

def process_chat_with_function_calls(messages, functions, available_functions, deployment_id):
  try:
    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto"
    )
    response_message = response["choices"][0]["message"]
    function_call = response_message.get("function_call")

    if function_call:
        print(f"Recommended Function call:\n{function_call}\n")
        function_name = function_call["name"]

        if function_name not in available_functions:
            return f"Function {function_name} does not exist"

        function_to_call = available_functions[function_name]
        function_args = json.loads(function_call["arguments"])

        if not check_args(function_to_call, function_args):
            return f"Invalid number of arguments for function: {function_name}"

        function_response = function_to_call(**function_args)

        print(f"Output of function call:\n{function_response}\n")

        messages.extend([
            {"role": response_message["role"], "name": function_name, "content":  str(function_args)}, # we convert to string because we have a dictionary of args, not just one.
            {"role": "function", "name": function_name, "content": function_response},
        ])

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )

        return second_response
  except Exception as e:
    print(e)


In [41]:
messages = [{"role": "user", "content": "I'm a silver member and I have 1200 points. Also I got that bonus 800 points in the mail last week. How many bucks do I have to spend please? "}]
assistant_response = process_chat_with_function_calls(messages, functions, available_functions, deployment_id)
#print(assistant_response['choices'][0]['message'])
print(assistant_response)

Recommended Function call:
{
  "name": "calculate_openai_bucks",
  "arguments": "{\n  \"tier\": \"silver\",\n  \"points\": \"2000\"\n}"
}

Output of function call:
$4000.00

Messages in second request:
{'role': 'user', 'content': "I'm a silver member and I have 1200 points. Also I got that bonus 800 points in the mail last week. How many bucks do I have to spend please? "}
{'role': 'assistant', 'name': 'calculate_openai_bucks', 'content': "{'tier': 'silver', 'points': '2000'}"}
{'role': 'function', 'name': 'calculate_openai_bucks', 'content': '$4000.00'}

{
  "id": "chatcmpl-7fMfMEbPVQtP6uqrOLMHTUh3eQbZX",
  "object": "chat.completion",
  "created": 1690092932,
  "model": "gpt-35-turbo-16k",
  "prompt_annotations": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
         